# Prophet Forecasting Model

This notebook implements a Facebook Prophet model on the `mock_kaggle.csv` dataset to generate demand forecasts.

In [ ]:
# --- Install and imports ---
!pip -q install prophet

import pandas as pd
import matplotlib.pyplot as plt
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# --- Load data ---
RAW_URL = "https://raw.githubusercontent.com/MiltosTsir/supply-chain-analysis-portfolio/main/portfolio/01-demand-forecasting/data/raw/mock_kaggle.csv"
df = pd.read_csv(RAW_URL, parse_dates=["data"])
df = df.sort_values("data").reset_index(drop=True)

# Prophet requires columns: ds (date), y (target)
prophet_df = df.rename(columns={"data":"ds", "venda":"y"})
prophet_df = prophet_df[["ds", "y"]]
prophet_df.head()

In [ ]:
# --- Train/Test split ---
horizon = 60 if len(prophet_df) > 120 else max(1, len(prophet_df)//5)
train = prophet_df.iloc[:-horizon]
test = prophet_df.iloc[-horizon:]
print("Train size:", len(train), " Test size:", len(test))

In [ ]:
# --- Fit Prophet model ---
model = Prophet(daily_seasonality=True, yearly_seasonality=True)
model.fit(train)

# --- Forecast ---
future = model.make_future_dataframe(periods=horizon)
forecast = model.predict(future)

fig1 = model.plot(forecast)
plt.title("Prophet Forecast")
plt.show()

fig2 = model.plot_components(forecast)
plt.show()

In [ ]:
# --- Evaluation ---
pred = forecast.set_index("ds").loc[test["ds"]]["yhat"]
mae = mean_absolute_error(test["y"], pred)
rmse = np.sqrt(mean_squared_error(test["y"], pred))

print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")